# 引入依赖

In [ ]:
import pandas as pd

from transformers import AutoTokenizer

import torch
from torch.utils.data import DataLoader, Dataset

from transformers import AutoConfig, AutoModelForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

# 模拟训练部分数据

In [2]:
data_list = [['苹果13/12/11/promax钢化膜13pro/12pro防窥iphone/x/xr/6/7皇迎 【黑边钻石防爆-高清透明膜】2片. iPhone X'],
 ['同福茂 适用华为畅享20Pro摄像头玻璃镜片畅享Z前后置大小照相机头 【】畅享20Z后主摄像头'],
 ['小米（MI）通用mp3MP4播放器迷你qq音乐学生运动跑步随身听电子书录音笔外放苹果 蓝色(带外放+可插内存卡) 8GB  王者套餐'],
 ['官方旗舰华强北智能watch6太空人s6代apple手表se顶配男士iwatch运动型iphone手 44MM/蓝高配版【太空人/长续航/双按键/高清 套餐二“送回环表带“'],
 ['峰爵苹果13promax手机壳平安喜乐新款iPhone13虎年创意女款13pro全包本命年小羊皮防摔 苹果13-红色-日富一日老虎'],
 ['WiFi收钱提示音响二维码语音播报器收款付款到账蓝牙小音箱色 炫酷黑 【蓝牙版】超长待机45天'],
 ['常用安全警示全套标示牌安全标识牌车间施工生产警告标志牌提示贴标语严禁烟火禁止吸烟有电危险标牌定制 禁止攀爬 20x30cm'],
 ['QYHJD 适用荣耀9i背夹充电宝华为8/9/10青春版背夹电池nova青春版电源薄 荣耀10青春版宝石蓝两万毫安 送耳机+数据线'],
 ['华为同款佳mp3mp4音乐播放器有屏学生随身听苹果风P5外放OTG可爱迷你 红 【带外放】可插卡 64GB  HiFi耳机套餐【鎹彩膜贴纸】'],
 ['2021华为matepadpro保护套12.6带笔槽10.8寸matepad11平板电脑壳V7硅胶p 【炭灰色+钢化膜-笔槽三折款】 华为平板 C5(10.4英寸)'],
 ['韵果【小学初中高中生学习专用】蓝牙mp3英语听力随身听学生版小型便携式mp4mp5音乐播放器听读 2英寸 触屏【+品质耳机+蓝牙功能】 16G蓝牙版(蓝牙功能+帮下载资料+发声词典)'],
 ['食掌门 绞肉机800瓦大功率小型肉馅机电动灌肠机'],
 ['ABS A4机柜文件夹盒PS柜配电箱威图柜WJ-1PS资料袋AE箱WJ-2文件夹 橙色带胶200个起'],
 ['鹿纯红米k40手机壳k40pro新款液态硅胶k40pro+全包防摔潮男女款软壳k40游戏增强版直边简 红米K40游戏增强版【海军蓝】*+钢化膜'],
 ['【官方原装】荧阙 适用2021款iPad保护套10.2英英寸平板9代8电脑Air4/3带笔槽ipad 【笔槽书本款】 梦想成真 付费软件APP iPad mini6(8.3英寸)'],
 ['机旺马华为畅享20plus手机壳磨砂超薄硅胶全包防摔欧美潮牌个性创意卡通图案男士款女网红软壳保护套 巧克联名 畅享20plus【加网红挂绳】+全屏膜'],
 ['OPPOReno5手机壳男金属镜头全包reno5k磨砂OPPOreno5手机套5G誉科创SN84 黑红色【单壳】刀锋 oppo Reno5'],
 ['源由 游戏键盘鼠标吃鸡oppoA59/A77/A83/A73/79/R15梦境手机壳保护皮套 升级版(黑色)键盘皮套+鼠标'],
 ['ITDK充电器插头超级快充数据充电线 适用于 【Max40W超级快充】充电器+1.5米5A数据线 华为Mate20 Pro/Mate20X 5g4g'],
 ['抖音快手热门歌曲音乐播放器迷你运动随身听mp4学生MP3 小怪兽 外响版+带卡含200首抖音+四件套'],
 ['胜枫 华为p50pro手机壳车载磁吸款p50镜头全包防摔保护套典藏版磨砂防指纹硬壳纯色极简保护套 P50【岩砂黑+车载支架】磁吸套装']]

        
data_df = pd.DataFrame(data_list, columns=['text'])
data_df = data_df.sample(frac=1)
data_df

,text
9,2021华为matepadpro保护套12.6带笔槽10.8寸matepad11平板电脑壳V...
11,食掌门 绞肉机800瓦大功率小型肉馅机电动灌肠机
5,WiFi收钱提示音响二维码语音播报器收款付款到账蓝牙小音箱色 炫酷黑 【蓝牙版】超长待机45天
1,同福茂 适用华为畅享20Pro摄像头玻璃镜片畅享Z前后置大小照相机头 【】畅享20Z后主摄像头
4,峰爵苹果13promax手机壳平安喜乐新款iPhone13虎年创意女款13pro全包本命年小...
8,华为同款佳mp3mp4音乐播放器有屏学生随身听苹果风P5外放OTG可爱迷你 红 【带外放】可...
17,源由 游戏键盘鼠标吃鸡oppoA59/A77/A83/A73/79/R15梦境手机壳保护皮套...
7,QYHJD 适用荣耀9i背夹充电宝华为8/9/10青春版背夹电池nova青春版电源薄 荣耀1...
12,ABS A4机柜文件夹盒PS柜配电箱威图柜WJ-1PS资料袋AE箱WJ-2文件夹 橙色带胶2...
19,抖音快手热门歌曲音乐播放器迷你运动随身听mp4学生MP3 小怪兽 外响版+带卡含200首抖音...


# 定义预训练模型&创建dataloader

In [3]:
'''
    定义BERT模型
'''
# model_checkpoint = 'chinese-roberta-wwm-ext'
# tokenizer_checkpoint = 'chinese-roberta-wwm-ext'
'''
    本地引入模型
'''
model_checkpoint = 'D:/env/bert_model/hfl/chinese-roberta-wwm-ext'
tokenizer_checkpoint = 'D:/env/bert_model/hfl/chinese-roberta-wwm-ext'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
# 简单自定义dataset
class PreTrainDataset(Dataset):
    def __init__(self, data_list, tokenizer, max_seq_len):
        super(PreTrainDataset, self).__init__()
        self.data_list = data_list
        self.len = len(data_list)
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __getitem__(self, index):
        example = self.data_list[index]
        data = self.tokenizer.encode_plus(example, return_token_type_ids=True, return_attention_mask=True, padding = 'max_length', max_length=self.max_seq_len)
        return {'input_ids':  torch.tensor(data['input_ids'][:self.max_seq_len], dtype=torch.long), 
                'token_type_ids':  torch.tensor(data['token_type_ids'][:self.max_seq_len], dtype=torch.long), 
                'attention_mask':  torch.tensor(data['attention_mask'][:self.max_seq_len], dtype=torch.long)}
    
    def __len__(self):
        return self.len

# 生成dataset
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
test_size = int(len(data_df)*0.8)
train_dataset = PreTrainDataset(data_df[:test_size]['text'].tolist(), tokenizer, 128)
test_dataset = PreTrainDataset(data_df[test_size:]['text'].tolist(), tokenizer, 128)
# 生成dataloader
train_dataloader = DataLoader(train_dataset, batch_size=8, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=8, num_workers=0)


# 加载定义模型&定义trainer

In [5]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 加载模型
config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)

model.to(device)
# 定义trainer
training_args = TrainingArguments(
                          output_dir='pretrain_bert',
                          overwrite_output_dir=True,
                          do_train=True, 
                          do_eval=True,
                          per_device_train_batch_size=32,
                          per_device_eval_batch_size=100,
                          evaluation_strategy='steps',
                          logging_steps=10000,
                          eval_steps = None,
                          prediction_loss_only=True,
                          learning_rate = 1e-5,
                          weight_decay=0.01,
                          adam_epsilon = 1e-8,
                          max_grad_norm = 1.0,
                          num_train_epochs = 10,
                          save_steps = -1,
                        push_to_hub=False
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

# 训练

In [ ]:
trainer.train()
trainer.save_model()